The following Worksheet is meant to display several examples of hypercube graph associahedra. This block of code defines the HypercubeGraph object, which stores a graph on vertices $\pm[n]$, as well as some functions for defining simplex graph associahedra and some utility functions. Run this block, and then run the remaining blocks to see some plots of various hypercube graph associahedra.

In [ ]:
def full_edge_list(n):
    edgelist=[]
    for i in range(1,n+1):
        for j in range(i+1,n+1):
            edgelist.append((i,j))
            edgelist.append((i,-j))
            edgelist.append((-i,j))
            edgelist.append((-i,-j))
    return edgelist

class HypercubeGraph(object):
    def __init__(self,en,edgelist):
        self.G = Graph()
        self.n = en
        for i in (1..en):
            self.G.add_vertex(i)
            self.G.add_vertex(-(i))
        self.G.add_edges(edgelist)
    
    def get_graph(self):
        return self.G
    
    def get_n(self):
        return self.n
    
    def tube_iterator_1(self):
        subgraphs = self.G.connected_subgraph_iterator(vertices_only=True)
        for graph in subgraphs:
            isTube= True
            for i in range(1,self.n+1):
                if {i,-i}.issubset(graph):
                    isTube=False
            if isTube:
                yield graph
    
    def associahedron(self):
        tubes = self.tube_iterator_1()
        P = Polyhedron(ieqs = [self.inequality(tube) for tube in tubes])
        return P
    
    def custom_associahedron(self,distance):
        tubes = self.tube_iterator_1()
        P = Polyhedron(ieqs = [self.custom_inequality(tube,distance) for tube in tubes])
        return P
    
    def inequality(self,tube):
        ieq = [(len(tube)*3^(self.n-1) -3^(len(tube)-2))] #We swap positive and negative so we get correct orientation
        for i in range(1,self.n+1):
            if i in tube:
                ieq.append(-1)
            elif (-i) in tube:
                ieq.append(1)
            else:
                ieq.append(0)
        return ieq
    def custom_inequality(self,tube,distance):
        ieq = [distance(len(tube))]
        for i in range(1,self.n+1): #swapping positive and negative...
            if i in tube:
                ieq.append(-1)
            elif (-i) in tube:
                ieq.append(1)
            else:
                ieq.append(0)
        return ieq

def simplex_custom_inequality(tube,n,f):
    ieq = [f(len(tube))]
    for i in range(1,n+1):
        if i in tube:
            ieq.append(1)
        else:
            ieq.append(0)
    return ieq
def simplex_graph_associahedron(graph,f=lambda x:3-3^(-x)):
    n = len(graph.vertices())
    tubes = graph.connected_subgraph_iterator(vertices_only=True)
    ieq_list = [simplex_custom_inequality(tube,n,f) for tube in tubes]
    eqn_list = [[1 for i in range(0,n+1)]]
    P = Polyhedron(ieqs = ieq_list,eqns=eqn_list)
    return P

def niceplot(hg,f=lambda x: x-3^(x-3),txt=""): #Plots polytope with pleasing ratios, then outputs a pretty spinning
    hg.get_graph().show(layout='circular')     #plot in addition to showing the original hypercube graph.
    return text(txt,(0,0)).plot()+hg.custom_associahedron(f).plot(frame=false,wireframe='black',polygon=False,point=False,spin=True)

In [ ]:
G1 = HypercubeGraph(3,[(1,2),(2,3)])
G2 = HypercubeGraph(3,[(1,2),(2,3),(1,3),(-1,-2),(-2,-3),(-1,-3)])
G3 = HypercubeGraph(3,full_edge_list(3))
#Our first few polytopes

In [ ]:
niceplot(G1,txt="Associahedron")

In [ ]:
niceplot(G2,txt="Permutahedron")

In [ ]:
niceplot(G3,txt="B3 Permutahedron")

In [ ]:
niceplot(HypercubeGraph(3,[(1,2),(2,3),(-1,-2),(-2,-3)]),txt="Bi-Associahedron")

In [ ]:
niceplot(HypercubeGraph(4,full_edge_list(4)),lambda x: x - 3^(x-4),txt="B4 Permutahedron")

In [ ]:
niceplot(HypercubeGraph(3,[(1,2),(2,3),(3,-1),(-2,-3)]),txt="Cyclohedron (only n=3)")
#This is one of the only two constructions of the cyclohedron up to isomorphism in n=3. For n=4, we conjecture that
#the cyclohedron cannot be realized as a hypercube graph associahedron.

In [ ]:
niceplot(HypercubeGraph(3,[(1,-2),(2,-3)]),txt="Pell Numbers") #OEIS A002203
#Preliminary research has shown an isomorphism between the lattice of Pell Permutations 
#(Lattice of Sashes, Shirley Law 2014), and the lattice of the poset of regions of 
#the normal fan of this polytope, for dimensions <=7. Obviously very promising.

In [ ]:
niceplot(HypercubeGraph(3,[(1,-2),(2,-3),(3,-1)]),txt="Companion Pell Numbers") #OEIS A002203

In [ ]:
niceplot(HypercubeGraph(3,[(1,2),(2,3),(1,3)]),txt="Stellohedron")

In [ ]:
niceplot(HypercubeGraph(4,[(1,2),(2,3),(3,4)]),lambda x: x - 3^(x-4))
#Note that Sage default plotting makes a mess of 4D non-symmetric polytopes.